# Building a Spam Filter with Naive Bayes Algorithm
Spam is a common nuisance in the age of information. Today, I will build a spam filter using the multinomial Naive Bayes algorithm.

I will use a [dataset of 5,572 English, real, non-encoded SMS messages](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection) collected by Tiago A. Almeida and José María Gómez Hidalgo. In the dataset, the SMS messages are classified as either "ham" (legitimate) or "spam." The data has been collected from numerous sources: the [UK spam-reporting website Grumbletext](http://www.grumbletext.co.uk/), the [NUS SMS Corpus](http://www.comp.nus.edu.sg/%7Erpnlpir/downloads/corpora/smsCorpus/), [Caroline Tag's PhD thesis](http://etheses.bham.ac.uk/253/1/Tagg09PhD.pdf), and the [SMS Spam Corpus v.0.1 Big](http://www.esp.uem.es/jmgomez/smsspamcorpus/). Information on how the dataset was collected and from where can be found [here](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition).

In [1]:
import pandas as pd

sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

sms.shape

(5572, 2)

In [2]:
sms

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
sms.describe(include='all')

,Label,SMS
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [4]:
sms['Label'].notnull().value_counts()

True    5572
Name: Label, dtype: int64

In [5]:
nr_ham = sms[sms['Label'] == 'ham'].shape[0]
nr_messages = sms.shape[0]

pct_ham = nr_ham/nr_messages*100
pct_spam = 100 - pct_ham
print('% Ham  : ' + str(pct_ham))
print('% Spam : ' + str(pct_spam))

% Ham  : 86.59368269921033
% Spam : 13.406317300789667


## Creating the Training and Test Sets
I will use random sampling to generate representative training and test sets of the data. I will use the training set data to train the software to recognize spam/ham messages. I will use the test data to test how accurate the software's predictions are.

In [6]:
sms_random = sms.sample(frac=1)
eighty_pct = round(0.8*len(sms_random))

training_set = sms_random[:eighty_pct]
test_set = sms_random[eighty_pct:]

training_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)

In [7]:
# Ensuring the training set and test set are of similar proportion to the original

nr_ham_training = training_set[training_set['Label'] == 'ham'].shape[0]
nr_ham_test = test_set[test_set['Label'] == 'ham'].shape[0]

nr_messages_training = training_set.shape[0]
nr_messages_test = test_set.shape[0]

pct_ham_training = nr_ham_training/nr_messages_training*100
pct_spam_training = 100 - pct_ham_training

pct_ham_test = nr_ham_test/nr_messages_test*100
pct_spam_test = 100 - pct_ham_test
    
print('Training set ham % : ' + str(pct_ham_training))
print('Test set ham %     : ' + str(pct_ham_test))

Training set ham % : 86.60834454912518
Test set ham %     : 86.53500897666068


## The Naive Bayes Algorithm
The Naive Bayes algorithm uses the rules of conditional probability to classify inputs. The relations for the Naive Bayes algorithm that I will use to classify messages in my spam filter are:

$$P(Ham)|w_{1},w_{2},...,w_{n}) \propto P(Ham) \cdot \prod_{i=1}^n P(w_{i}|Ham)$$

$$P(Spam)|w_{1},w_{2},...,w_{n}) \propto P(Spam) \cdot \prod_{i=1}^n P(w_{i}|Spam)$$

where:<br/>
&emsp;&emsp;$P(Ham) =$ the probability that a message is "ham" in the training set<br/>
&emsp;&emsp;$P(Spam) =$ the probability that a message is "spam" in the training set<br/>
&emsp;&emsp;$P(Ham)|w_{1},w_{2},...,w_{n}) =$ the probability that a message is "ham" based on each individual word in the message<br/>
&emsp;&emsp;$P(Spam)|w_{1},w_{2},...,w_{n}) =$ the probability that a message is "spam" based on each individual word in the message<br/>
&emsp;&emsp;$P(w_{i}|Ham) =$ the probability that word *i* is in a message, given that that message is "ham"<br/>
&emsp;&emsp;$P(w_{i}|Spam) =$ the probability that word *i* is in a message, given that that message is "spam"<br/>

In order to calculate $P(w_{i}|Ham)$ and $P(w_{i}|Spam)$, I will use these equations:

$$P(w_{i}|Ham) = \frac{N_{w_{i}|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{vocabulary}}$$

$$P(w_{i}|Spam) = \frac{N_{w_{i}|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{vocabulary}}$$

where:<br/>
&emsp;&emsp;$N_{w_{i}|Ham} =$ the number of times word *i* appears in the "ham" messages of the training set<br/>
&emsp;&emsp;$N_{w_{i}|Spam} =$ the number of times word *i* appears in the "spam" messages of the training set<br/>
&emsp;&emsp;$N_{Ham} =$ the number of "ham" messages there are in the training set<br/>
&emsp;&emsp;$N_{Spam} =$ the number of "spam" messages there are in the training set<br/>
&emsp;&emsp;$N_{vocabulary} =$ the number of unique words that exist in the training set vocabulary<br/>
&emsp;&emsp;$\alpha =$ a smoothing parameter<br/>

To solve these equations, I first must calculate the value of the variables.

To do that, I will split each training message into its individual words, generate a vocabulary list comprised of all unique words in the training set, and calculate each word count per message in the training set.

In [8]:
# Splitting each training message into its individual words

pd.options.mode.chained_assignment = None #Disabling the SettingWithCopy warning since it is benign in this case

training_set['SMS'] = training_set['SMS'].str.replace('\W',' ').str.lower().str.split().copy()

pd.options.mode.chained_assignment = 'warn' #Re-enabling the SettingWithCopy warning

In [9]:
# Generating a unique vocabulary list

vocab = set()

for message in training_set['SMS']:
    for word in message:
        if word not in vocab:
            vocab.add(word)
            
vocab = list(vocab)

In [10]:
# Calculating each word count per training set message

word_counts_per_sms = {word: [0] * len(training_set['SMS']) for word in vocab}
    
for index, message in enumerate(training_set['SMS']):
    for word in message:
        word_counts_per_sms[word][index] += 1
        
word_counts_per_sms = pd.DataFrame(word_counts_per_sms)

In [11]:
# Combining the training dataset with the word counts dataset

training_set_word_count = pd.concat([training_set, word_counts_per_sms], axis=1)

### Calculating the Variables
Now that my data is prepared, I am ready to calculate the variables for my multinomial Naive Bayes algorithm.

In [12]:
training_ham = training_set_word_count[training_set_word_count['Label'] == 'ham']
training_spam = training_set_word_count[training_set_word_count['Label'] == 'spam']

p_ham = pct_ham_training/100
p_spam = 1 - p_ham

n_ham = training_ham['SMS'].apply(len).sum()
n_spam = training_spam['SMS'].apply(len).sum()
        
n_vocab = len(vocab)

alpha = 1 #Laplace smoothing

In [13]:
p_wi_ham = {word:0 for word in vocab}
p_wi_spam = {word:0 for word in vocab}

for key in p_wi_ham:
    n_wi_ham = training_ham[key].sum()
    p_wi_ham[key] = (n_wi_ham + alpha)/(n_ham + alpha * n_vocab)
    
for key in p_wi_spam:
    n_wi_spam = training_spam[key].sum()
    p_wi_spam[key] = (n_wi_spam + alpha)/(n_spam + alpha * n_vocab)

### The Naive Bayes Algorithm
For the Naive Bayes algorithm, I will create a function that takes in an SMS message. The function will parse the message into separate words, calculate $P(Ham)|w_{1},w_{2},...,w_{n})$ and $P(Spam)|w_{1},w_{2},...,w_{n})$, print the value of each, compare the two, and then print whether the message is "ham" or "spam."

In [14]:
import re

def classify(message):
    
    re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_ham_given_message = p_ham
    p_spam_given_message = p_spam
    
    for word in message:
        if word in p_wi_ham:
            p_ham_given_message *= p_wi_ham[word]
        if word in p_wi_spam:
            p_spam_given_message *= p_wi_spam[word]
    
    print('P(Ham|message):', p_ham_given_message)
    print('P(Spam|message):', p_spam_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham!')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam!')
    else:
        print('equal probabilities, have a human classify this!')

In [15]:
# Testing the message with known inputs

test_message_1 = 'WINNER!! This is the secret code to unlock the money: C3421.' #Spam message
test_message_2 = 'Sounds good, Tom, then see u there' #Ham message

classify(test_message_1)
classify(test_message_2)

P(Ham|message): 2.0590497250330344e-19
P(Spam|message): 1.2053834559792751e-18
Label: Spam!
P(Ham|message): 3.711313692213348e-14
P(Spam|message): 8.972764800041065e-18
Label: Ham!


## Testing the Test Set
I will now alter the above Naive Bayes algorithm so that it prints nothing and instead returns the classification value of the message. Then, I will run the function on each message in the test set and store its value in a new column of the test set.

In [16]:
def classify_test_set(message):
    
    re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_ham_given_message = p_ham
    p_spam_given_message = p_spam
    
    for word in message:
        if word in p_wi_ham:
            p_ham_given_message *= p_wi_ham[word]
        if word in p_wi_spam:
            p_spam_given_message *= p_wi_spam[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [17]:
pd.options.mode.chained_assignment = None #Disabling the SettingWithCopy warning since it is benign in this case

test_set['predicted'] = test_set['SMS'].apply(classify_test_set) #Creating new test set column with predicted values

pd.options.mode.chained_assignment = 'warn' #Re-enabling the SettingWithCopy warning

test_set.head()

,Label,SMS,predicted
0,ham,In that case I guess I'll see you at campus lodge,ham
1,ham,"We are hoping to get away by 7, from Langport....",ham
2,ham,Aiyo please ü got time meh.,ham
3,ham,No you'll just get a headache trying to figure...,ham
4,ham,Ok i'm coming home now.,ham


### Checking for Accuracy
I will now check the accuracy of my Naive Bayes spam filter.

In [18]:
correct = 0 
total = len(test_set)

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
accuracy = correct/total*100

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy: ' + str(accuracy) + '%')

Correct: 1095
Incorrect: 19
Accuracy: 98.29443447037703%


## Conclusion


My multinomial Naive Bayes algorithm spam filter is 98.3% accurate for this training/test set. My spam filter checked 1,114 test messages and correctly classified 1,095 of these messages. This is good accuracy, since a spam filter that simply assumes every message is "ham" would have an accuracy of about 86%.

### Next Steps
* Review the incorrectly classified messages. If any are ham, find out if they all share something in common that caused them to be misclassified.